In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import ipywidgets as widgets
from IPython.display import display, clear_output
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import kendalltau, gumbel_r
from copulas.bivariate import Gumbel
import math


class RankDependencyAnalyzer:
    """
    Fixed Rank Dependency Analyzer for visualization.
    
    Properly models:
    - Concordance (Kendall's W): Overall agreement
    - Concurrence (Mutual Information): Shared information
    - Extremeness (Gumbel theta): Tail dependence
    """
    
    def __init__(self, num_samples=10000, random_seed=42):
        self.num_samples = num_samples
        np.random.seed(random_seed)

    def estimate_gumbel_theta(self, rankings):
        """Estimate Gumbel theta from rankings: theta = 1/(1-tau)"""
        taus = [kendalltau(rankings[i], rankings[j])[0] 
                for i in range(len(rankings)) 
                for j in range(i+1, len(rankings))]
        tau = np.median(taus)
        
        # Gumbel constraint: theta >= 1
        if tau >= 0.99:
            theta = 100.0
        elif tau <= 0:
            theta = 1.0
        else:
            theta = 1.0 / (1.0 - tau)
        
        return np.clip(theta, 1.0, 50.0)

    def generate_copula_samples(self, theta, W, MI):
        """
        Generate copula samples representing CDEF components.
        
        Args:
            theta: Gumbel parameter (extremeness/tail dependence)
            W: Kendall's W (concordance, 0-1)
            MI: Mutual Information (concurrence, 0+)
        
        Returns:
            U1, U2, U3: Three uniform marginals representing the components
        """
        # Component 1: Concordance (Kendall's W)
        # Model as correlation between raters - higher W = more correlation
        mean = [0, 0]
        cov = [[1, W], [W, 1]]  # Correlation = W
        X_concordance = np.random.multivariate_normal(mean, cov, self.num_samples)
        U1 = stats.norm.cdf(X_concordance[:, 0])
        
        # Component 2: Concurrence (Mutual Information)
        # Model as shared vs independent information
        # Higher MI = more shared structure
        if MI > 0.5:
            # High MI: significant shared information
            shared = np.random.uniform(0, 1, self.num_samples)
            independent = np.random.uniform(0, 1, self.num_samples)
            # Weight by MI (normalized)
            MI_weight = np.clip(MI / 3.0, 0, 1)  # Normalize MI to [0,1]
            U2 = MI_weight * shared + (1 - MI_weight) * independent
        else:
            # Low MI: mostly independent
            U2 = np.random.uniform(0, 1, self.num_samples)
        
        # Component 3: Extremeness (Gumbel theta)
        # Use actual Gumbel copula for tail dependence
        copula = Gumbel()
        copula.theta = theta
        
        # Generate from Gumbel copula (upper-tail dependence)
        # Use the copula's sampling method
        try:
            gumbel_samples = copula.sample(self.num_samples)
            U3 = gumbel_samples[:, 0]  # Take first marginal
        except:
            # Fallback: use inverse transform method
            # Gumbel CDF: exp(-((-log u)^theta + (-log v)^theta)^(1/theta))
            u_base = np.random.uniform(0.001, 0.999, self.num_samples)
            v_base = np.random.uniform(0.001, 0.999, self.num_samples)
            
            # Apply Gumbel transformation
            log_u = -np.log(u_base)
            log_v = -np.log(v_base)
            U3 = np.exp(-(log_u**theta + log_v**theta)**(1/theta))
        
        return U1, U2, U3


def generate_copula_wireframe(theta, W, MI, N=30):
    """
    Generate wireframe for 3D copula visualization.
    
    Args:
        theta: Gumbel parameter (extremeness)
        W: Kendall's W (concordance)
        MI: Mutual Information (concurrence)
        N: Grid resolution
    """
    analyzer = RankDependencyAnalyzer(num_samples=10000)
    U1, U2, U3 = analyzer.generate_copula_samples(theta, W, MI)

    # Create grid
    u = np.linspace(0.01, 0.99, N)
    v = np.linspace(0.01, 0.99, N)
    U, V = np.meshgrid(u, v)

    # Estimate joint density on grid
    U_exp = U[np.newaxis, :, :]
    V_exp = V[np.newaxis, :, :]

    sample_1_exp = U1[:, np.newaxis, np.newaxis]
    sample_2_exp = U2[:, np.newaxis, np.newaxis]

    # Empirical copula: proportion of samples <= (u,v)
    Z = np.mean((sample_1_exp <= U_exp) & (sample_2_exp <= V_exp), axis=0)
    
    # Weight by extremeness (U3) to show tail dependence
    extremeness_weight = np.mean(U3) if theta > 1.5 else 1.0
    Z = Z * extremeness_weight

    return U, V, Z


# Initial Values (based on your real data)
initial_theta = 5.21  # Your data's scaled theta
initial_W = 0.85      # Your data's Kendall's W
initial_MI = 1.266    # Your data's MI
initial_elev = 30
initial_azim = 45

# Interactive Widgets
theta_slider = widgets.FloatSlider(
    value=initial_theta, min=1.0, max=30.0, step=0.5, 
    description='Extremeness (θ)',
    style={'description_width': 'initial'}
)
W_slider = widgets.FloatSlider(
    value=initial_W, min=0.0, max=1.0, step=0.05, 
    description='Concordance (W)',
    style={'description_width': 'initial'}
)
MI_slider = widgets.FloatSlider(
    value=initial_MI, min=0.0, max=3.0, step=0.1, 
    description='Concurrence (MI)',
    style={'description_width': 'initial'}
)
elev_slider = widgets.IntSlider(
    value=initial_elev, min=-90, max=90, step=5, 
    description='Elevation'
)
azim_slider = widgets.IntSlider(
    value=initial_azim, min=0, max=360, step=5, 
    description='Azimuth'
)

# Preset buttons for scenarios
preset_buttons = widgets.ToggleButtons(
    options=[
        ('Your Data (Genuine)', (5.21, 0.85, 1.266)),
        ('Phantom', (30.7, 0.96, 2.358)),
        ('Strong Genuine', (4.2, 0.75, 1.611)),
        ('Random', (1.3, 0.27, 0.59))
    ],
    description='Presets:',
    style={'description_width': 'initial'}
)


def update_plot(theta, W, MI, elev, azim):
    """Update the 3D wireframe plot"""
    clear_output(wait=True)

    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    U, V, Z = generate_copula_wireframe(theta, W, MI)

    # Create wireframe
    ax.plot_wireframe(U, V, Z, color='blue', alpha=0.6, linewidth=0.5)
    
    # Add surface for better visualization
    surf = ax.plot_surface(U, V, Z, cmap='viridis', alpha=0.3, 
                           edgecolor='none', antialiased=True)
    
    ax.set_xlabel("Concordance (Kendall's W)", fontsize=10)
    ax.set_ylabel("Concurrence (Mutual Information)", fontsize=10)
    ax.set_zlabel("Joint Density (weighted by θ)", fontsize=10)
    
    # Interpret the scenario
    if theta > 15 and W > 0.85:
        scenario = "⚠️ PHANTOM: Shared extreme biases"
        color = 'red'
    elif W > 0.65 and 2.5 < theta < 6.5:
        scenario = "✓ GENUINE: Natural agreement"
        color = 'green'
    elif W < 0.4:
        scenario = "○ RANDOM: No systematic agreement"
        color = 'gray'
    else:
        scenario = "→ MIXED: Moderate agreement"
        color = 'orange'
    
    title = f"CDEF Copula Visualization\n"
    title += f"θ={theta:.2f}, W={W:.2f}, MI={MI:.2f}\n"
    title += f"{scenario}"
    
    ax.set_title(title, fontsize=12, color=color, weight='bold')
    ax.view_init(elev=elev, azim=azim)
    
    # Add colorbar
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)
    
    plt.tight_layout()
    plt.show()
    
    # Display relative importance
    total = W + MI + theta
    print(f"\nRelative Importance (NOT probabilities):")
    print(f"  Concordance (W):  {W/total:.3f} ({W/total*100:.1f}%)")
    print(f"  Concurrence (MI): {MI/total:.3f} ({MI/total*100:.1f}%)")
    print(f"  Extremeness (θ):  {theta/total:.3f} ({theta/total*100:.1f}%)")
    print(f"\nInterpretation:")
    if theta/total > 0.6:
        print("  → Tail dependence DOMINATES (agreement on extremes)")
    elif W/total > 0.5:
        print("  → Global concordance DOMINATES (uniform agreement)")
    else:
        print("  → Balanced contribution from all factors")


def apply_preset(change):
    """Apply preset values when button is clicked"""
    theta, W, MI = preset_buttons.value
    theta_slider.value = theta
    W_slider.value = W
    MI_slider.value = MI


preset_buttons.observe(apply_preset, names='value')

# Display Widgets and Plot
ui = widgets.VBox([
    widgets.HTML("<h3>CDEF Copula Visualization</h3>"),
    widgets.HTML("<p><b>Adjust parameters or select presets:</b></p>"),
    preset_buttons,
    widgets.HTML("<p><b>Manual controls:</b></p>"),
    theta_slider, 
    W_slider, 
    MI_slider,
    widgets.HTML("<p><b>View angle:</b></p>"),
    elev_slider, 
    azim_slider
])

output = widgets.interactive_output(update_plot, {
    'theta': theta_slider,
    'W': W_slider,
    'MI': MI_slider,
    'elev': elev_slider,
    'azim': azim_slider
})

display(ui, output)
